# **Import**

In [0]:
from google.colab import drive
import tensorflow as tf
import numpy
import random
import pickle
import cv2
import os

# **Mount Cloud Drive**

In [2]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# **Load Model**

In [3]:
model = tf.keras.models.load_model('/content/drive/My Drive/model8.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


# **Load Data from Images**

In [0]:
folders = [0, 30, 60, 90, 120, 150, 180, -30, -60, -90, -120, -150]
CLASSES = [0,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,-10,-20,-30,-40,-50,-60,-70,-80,-90,-100,-110,-120,-130,-140,-150,-160,-170]
evaluation = []
X_ev = []
y_ev = []


for folder in folders:
  path = '/content/drive/My Drive/ImageEvaluations/ImageEvaluation_EyeFaceRelativePosition'
  path = os.path.join(path, str(folder))
  index = CLASSES.index(folder)
  for image in os.listdir(path):
    img = cv2.imread(os.path.join(path, image), 0)
    img = cv2.resize(img/255, (100,100))
    evaluation.append([img, index])


random.shuffle(evaluation)


for img, label in evaluation:
  X_ev.append(img)
  y_ev.append(label)

# **Process Data Loaded**

In [0]:
length = len(X_ev)


X_ev = numpy.array(X_ev).reshape(length, 100, 100, 1)


y_evaluation = [None]*length
for i in range(length):
  li = [0]*36
  index = y_ev[i]
  li[index] = 1
  y_evaluation[i] = li
y_ev = y_evaluation


y_ev = numpy.array(y_ev).reshape(length, 36)

# **Pickle Data onto Cloud Drive**

In [0]:
out = open('/content/drive/My Drive/Data/X_ev_EyeFaceRelativePosition.pickle', 'wb')
pickle.dump(X_ev, out)
out.close

out = open('/content/drive/My Drive/Data/y_ev_EyeFaceRelativePosition.pickle', 'wb')
pickle.dump(y_ev, out)
out.close

<function BufferedWriter.close>

# **Load Data from Pickle**

In [0]:
X_ev_pickle = open('/content/drive/My Drive/Data/X_ev_EyeFaceRelativePosition.pickle', 'rb')
X_ev = pickle.load(X_ev_pickle)

y_ev_pickle = open('/content/drive/My Drive/Data/y_ev_EyeFaceRelativePosition.pickle', 'rb')
y_ev = pickle.load(y_ev_pickle)

# **Model Evaluation**

In [0]:
loss, accuracy = model.evaluate(X_ev, y_ev)
print(loss)
print(accuracy)

324/324 [==============================] - 4s 11ms/sample - loss: 67.8383 - acc: 0.0000e+00
67.83826724393869
0.0


# **Accuracy for Each Class**

Tolerance is a variable defined to limit the roughness of an accurate prediction. For example, for an image of class 60, a tolerance equal to 20 would mean that predictions in [40, 50, 60, 70, 80] would all be correct predictions.

In [0]:
CLASSES = [0,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,-10,-20,-30,-40,-50,-60,-70,-80,-90,-100,-110,-120,-130,-140,-150,-160,-170]
tolerance = 2


def normalise(value):
  if value > 180:
    return value - 360
  elif value <= -180:
    return value + 360
  else:
    return value   


def in_range(actual, expected, scale):
  li = [expected]
  for i in range(scale+1):
    value1 = normalise(expected + 10*i)
    value2 = normalise(expected - 10*i)
    li = li + [value1]
    li = li + [value2]
  if actual in li:
    return True
  else:
    return False


def class_accuracy(folder):
  correct = 0
  false = 0
  path = '/content/drive/My Drive/ImageEvaluations/ImageEvaluation_EyeFaceRelativePosition'
  path = os.path.join(path, str(folder))
  wrong_images = []
  for image in os.listdir(path):
    img = cv2.imread(os.path.join(path, image), 0)
    img = cv2.resize(img/255, (100,100))
    img = numpy.array(img).reshape(1, 100, 100, 1)
    predictions = model.predict(img)[0].tolist()
    index = predictions.index(max(predictions))
    if in_range(CLASSES[index], folder, tolerance):
      correct += 1
    else:
      false += 1
      wrong_images.append(os.path.join(str(folder), image))
  accuracy = correct/(correct+false)
  return correct, false, wrong_images

In [7]:
folders = [0, 30, 60, 90, 120, 150, 180, -30, -60, -90, -120, -150]
wrong_predictions = []
total_correct = 0
total_false = 0


for folder in folders:
  correct, false, wrong_images = class_accuracy(folder)
  total_correct = total_correct + correct
  total_false = total_false + false
  print(folder, correct/(correct+false))
  wrong_predictions = wrong_predictions + wrong_images
print('overall', total_correct/(total_correct+total_false))

0 0.42105263157894735
30 0.4
60 0.0
90 0.0
120 0.0
150 0.0
180 0.0
-30 0.7021276595744681
-60 0.5666666666666667
-90 0.21875
-120 0.0
-150 0.08
overall 0.23148148148148148
